In [33]:
#read in muse_v3.csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('muse_v3.csv')
df.head()

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre
0,.,'Till I Collapse,Eminem,['aggressive'],6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap
1,https://www.last.fm/music/metallica/_/st.%2banger,St. Anger,Metallica,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin',Rick Ross,['aggressive'],1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,M.I.A.,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,['aggressive'],7,3.771176,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal


In [34]:
#given song name and artist, scrape 

#check how many rows have nan for spotify_id
df['spotify_id'].isnull().sum()


28371

In [35]:
#drop rows with nan for spotify_id

df = df.dropna(subset=['spotify_id'])
df['spotify_id'].isnull().sum()


0

In [4]:

GENIUS_API_TOKEN = 'osvPFwS1hvo1Y0oUI659YobdX11IWnS-B8-3uEw-SxaIllHqzmdBtCRoH8jzXlNh'

import requests
from bs4 import BeautifulSoup
import os
import re


In [ ]:
#only keep alphanumeric characters and spaces in song name
df['track'] = df['track'].str.replace('[^a-zA-Z0-9\s]', '')
df['artist'] = df['artist'].str.replace('[^a-zA-Z0-9\s]', '')
df

In [2]:
def scrape_lyrics(artist, song):
    artistname = artist.replace(' ', '-')
    songname = song.replace(' ', '-')
    url = 'https://genius.com/' + artistname + '-' + songname + '-lyrics'
    #print(url)
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find('div', class_='lyrics')
    lyrics2 = html.find('div', class_='Lyrics__Container-sc-1ynbvzw-5 Dzxov')

    
    if lyrics1:
        lyrics = lyrics1.get_text(separator=' ', strip=True)
    elif lyrics2:
        lyrics = lyrics2.get_text(separator=' ', strip=True)
    elif lyrics1 == lyrics2:
        return None
    
    #new line is missing from lyrics
    return lyrics
    




In [6]:
print(scrape_lyrics('rihanna', 'only girl in the world'))

[Intro] La-la-la-la La-la-la-la La-la-la-la (Uh, yeah) La-la-la-la [Verse 1] I want you to love me Like I'm a hot ride (Uh, yeah) Be thinking of me (Uh) Doing what you like (Haha) So, boy, forget about the world 'Cause it's gon' be me and you tonight (Yeah) I wanna make you beg for it Then I'ma make you swallow your pride, oh (Uh, uh) [Chorus] Want you to make me feel Like I'm the only girl in the world Like I'm the only one that you'll ever love Like I'm the only one who knows your heart Only girl in the world Like I'm the only one that's in command 'Cause I'm the only one who understands How to make you feel like a man, yeah Want you to make me feel Like I'm the only girl in the world Like I'm the only one that you'll ever love Like I'm the only one who knows your heart Only one [Verse 2] Want you to take it (Uh) Like a thief in the night (Uh) Hold me like a pillow Make me feel right (Uh) Baby, I'll tell you all my secrets that I'm keeping You can come inside (Yeah) And when you ente

In [42]:
def scrape_last_fm(url):
    url = url + '/+lyrics'
    try:
        page = requests.get(url)
    except:
        return None
    html = BeautifulSoup(page.text, 'html.parser')
    #span class = 'lyrics-body'
    lyrics = html.find_all('p', class_='lyrics-paragraph')
    if lyrics:
        for i in range(len(lyrics)):
            lyrics[i] = lyrics[i].get_text(separator=' ', strip=True)
        lyrics = ' '.join(lyrics)
    else:
        return None
    return lyrics

In [46]:

from tqdm import tqdm
indices = df.index.tolist()

for i in tqdm(indices):
    url = df['lastfm_url'][i]
    lyrics = scrape_last_fm(url)
    if(lyrics == None):
        artist = df['artist'][i]
        song = df['track'][i]
        lyrics = scrape_lyrics(artist, song)
    #write lyrics to csv
    #open file in append mode
    with open('muse_lyrics_lastfm.txt', 'a', encoding='utf-8') as f:
        if(lyrics == None):
            f.write(str(i) + ':None')
        else:
            f.write(str(i) + ':' + lyrics)
        f.write('\n')

In [48]:
#howe many unique seeds are there in df
indices = df.index.tolist()
seeds = []
for i in indices:
    seed = df['seeds'][i]
    seed_list = seed.split(',')
    for s in seed_list:
        #remove '[' and ']'
        s = s.replace('[', '')
        s = s.replace(']', '')
        #remove whitespace
        s = s.strip()
        #get rid of "'"
        s = s.replace("'", '')
        seeds.append(s)

seeds = list(set(seeds))
len(seeds)

seeds_to_index = {}
index_to_seeds = {}
for i in range(len(seeds)):
    seeds_to_index[seeds[i]] = i
    index_to_seeds[i] = seeds[i]

#create one hot encoding for seeds
seeds_encoded = []
for i in indices:
    encoding = np.zeros(len(seeds))
    seed = df['seeds'][i]
    seed_list = seed.split(',')
    for s in seed_list:
        #remove '[' and ']'
        s = s.replace('[', '')
        s = s.replace(']', '')
        #remove whitespace
        s = s.strip()
        #get rid of "'"
        s = s.replace("'", '')
        encoding[seeds_to_index[s]] = 1
    seeds_encoded.append(encoding)





In [49]:
df['seeds_encoded'] = seeds_encoded
df

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,seeds_encoded
0,.,Till I Collapse,Eminem,['aggressive'],6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,https://www.last.fm/music/metallica/_/st.%2banger,St Anger,Metallica,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin,Rick Ross,['aggressive'],1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,MIA,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,['aggressive'],7,3.771176,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
89983,https://www.last.fm/music/quietdrive/_/secret,Secret,Quietdrive,['transparent'],1,5.370000,3.450000,5.330000,909ba2e0-6076-4ad8-8be9-a335725de4ef,2bRIsZ92JRKlvQOZlyR9CO,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
89985,https://www.last.fm/music/medications/_/the%2b...,The Last of the Rest Was the End,Medications,['transparent'],1,5.370000,3.450000,5.330000,ffd10ae8-858a-4b5e-819e-64f1174bbd42,7o3Np7cho9cBCrNDokxzYC,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
89986,https://www.last.fm/music/daniel%2blanois/_/lo...,Lovechild,Daniel Lanois,['transparent'],2,6.685000,4.405000,5.625000,16c3d394-c4d4-4dc2-bbf1-b2bef3ac861c,4fVObxldDzxxRD6a5Eth9s,indie,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
89991,https://www.last.fm/music/tapage/_/last%2binhale,Last Inhale,Tapage,['transparent'],6,4.301667,3.341667,4.466667,b744fdbc-7b4c-49a1-a3d6-99047d82eb34,5WxwRwUQ4R4L46VEm3213y,ambient,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [50]:
#load in lyrics
lyrics = []
with open('muse_lyrics_lastfm.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        line = line.split(':')
        if(len(line) == 1):
            print(line)
            continue
        lyrics.append(' '.join(line[1:]))

['']
['']
['44']


In [51]:
len(lyrics)

61630

In [52]:
len(df)


61630

In [53]:
df['lyrics'] = lyrics

In [ ]:
#check how many rows have 'None' for lyrics-- 'None' is a string
df['lyrics'].value_counts()
#for all rows with 'None' for lyrics, try scraping again
df_none = df[df['lyrics'] == 'None']
print(df_none.index.tolist())


In [58]:
df


,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,seeds_encoded,lyrics
0,.,Till I Collapse,Eminem,['aggressive'],6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[Intro Eminem] 'Cause sometimes you just feel...
1,https://www.last.fm/music/metallica/_/st.%2banger,St Anger,Metallica,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",St. Anger 'round my neck St. Anger 'round my n...
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin,Rick Ross,['aggressive'],1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[Intro Rick Ross] Legendary Runners You know ...
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,MIA,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","Road runner, road runner Going hundred miles p..."
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,['aggressive'],7,3.771176,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",I don't need your forgiveness I don't need you...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89983,https://www.last.fm/music/quietdrive/_/secret,Secret,Quietdrive,['transparent'],1,5.370000,3.450000,5.330000,909ba2e0-6076-4ad8-8be9-a335725de4ef,2bRIsZ92JRKlvQOZlyR9CO,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","Hey, bride of mine I love you So why'd you fin..."
89985,https://www.last.fm/music/medications/_/the%2b...,The Last of the Rest Was the End,Medications,['transparent'],1,5.370000,3.450000,5.330000,ffd10ae8-858a-4b5e-819e-64f1174bbd42,7o3Np7cho9cBCrNDokxzYC,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Desperate days When solutions are sold For a p...
89986,https://www.last.fm/music/daniel%2blanois/_/lo...,Lovechild,Daniel Lanois,['transparent'],2,6.685000,4.405000,5.625000,16c3d394-c4d4-4dc2-bbf1-b2bef3ac861c,4fVObxldDzxxRD6a5Eth9s,indie,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Child Child Oh child Child Child trembling You...
89991,https://www.last.fm/music/tapage/_/last%2binhale,Last Inhale,Tapage,['transparent'],6,4.301667,3.341667,4.466667,b744fdbc-7b4c-49a1-a3d6-99047d82eb34,5WxwRwUQ4R4L46VEm3213y,ambient,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",None


In [61]:
indices_to_remove = [36463, 53049] 
df.drop(indices_to_remove, inplace=True)
df = df.reset_index(drop=True)


In [ ]:
#save df to csv
df.to_csv('muse_df_with_lyrics.csv', index=False)